This examples show how to use MiniSom to solve a classification problem. The classification mechanism will be implemented with MiniSom and the evaluation will make use of sklearn.

First, let's load a dataset (in this case the famous Iris dataset) and apply normalization:

In [4]:
from minisom import MiniSom
import numpy as np

data = np.genfromtxt('iris.csv', delimiter=',', usecols=(0, 1, 2, 3))
data = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, data)
labels = np.genfromtxt('iris.csv', delimiter=',', usecols=(4), dtype=str)

Here's naive classification function that classifies a sample in `data` using the label assigned to the associated winning neuron. A label $c$ is associated to a neuron if the majority of samples mapped in that neuron have label $c$. The function will assign the most common label in the dataset in case that a sample is mapped to a neuron for which no class is assigned.

In [2]:
def classify(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(X_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

Now we can 1) split the data in train and test set, 2) train the som, 3) print the classification report that contains all the metrics to evaluate the results of the classification.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(data, labels, stratify=labels)

som = MiniSom(7, 7, 4, sigma=3, learning_rate=0.5, 
              neighborhood_function='triangle', random_seed=10)
som.pca_weights_init(X_train)
som.train_random(X_train, 500, verbose=False)

print(classification_report(y_test, classify(som, X_test)))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       0.92      1.00      0.96        12
   virginica       1.00      0.92      0.96        13

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

